# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [9]:
# Your solution goes here
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import timedelta

In [10]:

def scrape_imdb(type,rate,date_start,date_end):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    url=f'https://www.imdb.com/search/title/?title_type={type}&user_rating={rate},&release_date={date_start},{date_end}'
    response = requests.get(url, headers=headers)
    response_status=response.status_code
    
    if response_status==200:
        print(f'response is Positive')
        
        response.headers['Content-Type']
        
        info= BeautifulSoup(response.content, "html.parser")
        
        # Number and title
        movies=[]
        nr=[]

        for item in info.find_all('h3'):
            movie=item.string
            movie=movie.split(" ",1)
            movies.append(movie[1])
            nr.append(movie[0])

        movies.pop(-1)
        nr.pop(-1)
        
        # Rating
        rating=[]
        for item in info.find_all(["div","span"], class_="ipc-rating-star--imdb"):
            rating.append(item.get("aria-label").replace("IMDb rating: ",""))

        # Year and duration of the movie
        year=[]
        duration=[]
        for item in info.find_all(["div","span"], class_="dli-title-metadata-item"):
            movie_time=item.string
            if len(movie_time)==4:
                year.append(movie_time)
            elif 'h' in movie_time:
                if 'm' in movie_time:
                    time='0'+movie_time.replace('h',":").replace('m',"").replace(" ","")
                    seconds_=timedelta(hours=int(time.split(':')[0]), minutes=int(time.split(':')[1]))
                    minutes_=seconds_.total_seconds()/60
                    duration.append(minutes_)
                elif 'm' not in movie_time:
                    time='0'+movie_time.replace('h',":").replace(" ","")
                    seconds_=timedelta(hours=int(time.split(':')[0]), minutes=int("00"))
                    minutes_=seconds_.total_seconds()/60
                    duration.append(minutes_)

        # Votes
        total_votes=[]
        for item in info.find_all(["div"], class_="jPoaKS"):
            votes=item.text
            votes=votes.replace("Votes","")
            total_votes.append(votes)

        #Description
        description_movie=[]
        for item in info.find_all("div", class_="ipc-html-content-inner-div"):
            description=item.string
            description_movie.append(description)

        # actors
        main_actors=[]
        for item in info.find_all(["img"]):
            actors=item.get('alt')
            main_actors.append(actors)

        main_actors.pop(1)
        main_actors.pop(-1)

        data_frame=pd.DataFrame({"movies" : movies, "nr_movie": nr, "rating":rating, "year": year, "duration":duration, "total_votes": total_votes,"description_movie":description_movie, "main_actors": main_actors})
        return data_frame       
    else:
        print(f'response is negative')
    




In [11]:
type="feature"
rate="7.5"
date_start="1990-01-01"
date_end="1992-12-31"

entry=scrape_imdb(type,rate,date_start,date_end)

response is Positive


In [12]:
entry

,movies,nr_movie,rating,year,duration,total_votes,description_movie,main_actors
0,Goodfellas,1.,8.7,1990,145.0,"1,250,607",The story of Henry Hill and his life in the ma...,
1,The Silence of the Lambs,2.,8.6,1991,118.0,"1,540,173",A young F.B.I. cadet must receive the help of ...,Jodie Foster in The Silence of the Lambs (1991)
2,My Cousin Vinny,3.,7.6,1992,120.0,"139,517",Two New Yorkers accused of murder in rural Ala...,"Joe Pesci, Marisa Tomei, and Fred Gwynne in My..."
3,Terminator 2: Judgment Day,4.,8.6,1991,137.0,"1,168,611","A cyborg, identical to the one who failed to k...",Arnold Schwarzenegger in Terminator 2: Judgmen...
4,Fried Green Tomatoes,5.,7.7,1991,130.0,"82,865","Evelyn, an ordinary housewife, visits a nursin...",Fried Green Tomatoes (1991)
5,Total Recall,6.,7.5,1990,113.0,"352,790",When a man goes in to have virtual vacation me...,Arnold Schwarzenegger in Total Recall (1990)
6,Dances with Wolves,7.,8.0,1990,181.0,"288,977","Lieutenant John Dunbar, assigned to a remote w...",Kevin Costner in Dances with Wolves (1990)
7,Reservoir Dogs,8.,8.3,1992,99.0,"1,083,263",When a simple jewelry heist goes horribly wron...,"Steve Buscemi, Harvey Keitel, Michael Madsen, ..."
8,Chaplin,9.,7.5,1992,143.0,"61,951",An elderly Charlie Chaplin discusses his autob...,Robert Downey Jr. in Chaplin (1992)
9,The Godfather Part III,10.,7.6,1990,162.0,"422,400","Follows Michael Corleone, now in his 60s, as h...","Al Pacino, Andy Garcia, Sofia Coppola, and Tal..."


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

# Your solution goes here
The page does not have pagination anymore, but a continue scroll